
## Del lado del servidor en la ecuación MCP

En el otro lado de la ecuación MCP, junto al cliente, se encuentra el servidor. En el Capítulo 3, los ejemplos de cliente hicieron uso de un servidor que le daba a la aplicación host la capacidad de realizar cálculos. Este servidor ya estaba preconstruido y se ejecutaba de manera local. En este capítulo, nuestros ejemplos mostrarán la construcción de este mismo servidor desde cero, incluyendo todas las **primitivas** y características que los servidores pueden ofrecer a los clientes y a las aplicaciones host. Comenzaremos con una visión general del servidor, incluyendo:

* Qué hacen los servidores y por qué son útiles
* Cómo se conectan a los clientes
* Las dos principales vías para construirlos usando el SDK de Python

Después, entraremos directamente en los detalles:

* **Primitivas del servidor** (tools, prompts y resources) y cómo servirlas
* **Utilidades del servidor** (completion, logging, notifications y pagination) y cómo usarlas
* **Seguridad del servidor**
* Uso de **recursos proporcionados por el cliente**, como sampling y elicitation

Como desarrollador de servidores, también necesitarás familiarizarte con las pruebas de tu servidor. Aprenderás sobre **MCP Inspector**, una herramienta para visualizar e interactuar con tu servidor y sus herramientas, así como sobre **evaluaciones**, pruebas usadas en sistemas de IA tradicionales que podemos reutilizar para nuestro propio uso.

---

## Servidores, ¿Para qué sirven?

Los servidores son el punto de entrada más popular para desarrolladores que comienzan con MCP. Herramientas populares como **Claude Desktop, Claude Code, Cursor**, y más, incluyen soporte incorporado para agregar servidores a sus aplicaciones, y los mantenedores de los distintos **SDKs de MCP** se han enfocado en facilitar al máximo el desarrollo de servidores para los desarrolladores. Esto ha tenido algunas consecuencias, como servidores inseguros o de bajo rendimiento que se construyen y se publican de inmediato, y el ocultamiento de algunos de los beneficios más profundos que los servidores MCP proporcionan.

Lo más obvio que hacen los servidores es **proporcionar a las aplicaciones acceso a las primitivas MCP**, otorgando nuevas capacidades que antes no tenían. Estas **primitivas** son los **bloques de construcción de MCP** y consisten en **tools, prompts y resources**. Los encontraste en el Capítulo 3, donde tu cliente integró estas **primitivas** con una aplicación de chat simple de la que formaba parte. Al usar estas **primitivas**, aprendiste lo que ofrecen:

* **Tools**: ofrecen acciones programadas
* **Resources**: ofrecen datos estructurados que pueden usarse como contexto por el modelo de lenguaje
* **Prompts**: ofrecen instrucciones interactivas para el modelo de lenguaje de la aplicación host

En la especificación MCP, cada **primitiva** está diseñada para ser controlada por una entidad diferente:

* **Tools**: controladas por el modelo que las invoca
* **Resources**: controladas por la aplicación que decide cómo usarlas
* **Prompts**: controladas por el usuario de la aplicación que activa su uso y proporciona los inputs

Esta lista de usos y controladores se resume en la siguiente tabla, reproducida con algunas modificaciones de la especificación MCP:

| Primitiva    | Controlador                 | Descripción                                                         |
| ------------ | --------------------------- | ------------------------------------------------------------------- |
| **Tool**     | El modelo                   | Funciones expuestas al LLM para realizar alguna acción              |
| **Resource** | La aplicación               | Datos contextuales gestionados por la aplicación usando el servidor |
| **Prompt**   | El usuario de la aplicación | Plantillas interactivas invocadas por elección del usuario          |

---

### La verdadera magia de los servidores

Lo realmente interesante de los servidores está en otro lugar, aunque sigue relacionado con cómo **aumentan las capacidades de las aplicaciones**: **resuelven el problema de la distribución**.

Antes de MCP, las herramientas, fuentes de datos y prompts estaban fuertemente acoplados a la aplicación inteligente que los usaba, lo que hacía difícil compartirlos y reutilizarlos. Por ejemplo, si alguien quería que el LLM de su aplicación tuviera acceso a la documentación de AWS, tendría que conectarse, escribir el código adecuado para obtenerla y luego ponerla a disposición del LLM de su aplicación.

Con MCP, alguien en AWS que conozca mejor la herramienta proporcionada puede escribir un **servidor MCP** que brinde esta funcionalidad a cualquiera que pueda conectarse a él. Ahora, los desarrolladores de aplicaciones pueden concentrarse en la lógica de su aplicación, dejando que los servidores MCP manejen el resto. Los creadores de servidores pueden distribuirlos a través de plataformas de compartición de código como GitHub o como un servidor en ejecución al que los clientes MCP puedan conectarse de manera remota.

Cuando un servidor MCP puede escribirse en un solo archivo, compartirlo y reutilizarlo se vuelve mucho más fácil, y aún más cuando el servidor se despliega en un servidor web remoto.

---



---

## Conectando Servidores a Aplicaciones Usando Transportes

La clave de cómo los servidores se comunican con las aplicaciones a través de clientes MCP es el **transporte**. Los encontraste en los ejemplos de cliente del Capítulo 3 y profundizarás en los detalles técnicos más adelante, pero para el desarrollo de servidores es importante entender las opciones disponibles para **distribuir tu servidor a aplicaciones cliente**.

El protocolo en sí utiliza **JSON-RPC** como protocolo de comunicación subyacente, y los transportes sirven para comunicar mensajes codificados con ese protocolo entre el cliente y el servidor. MCP y sus SDKs soportan dos transportes principales:

* **stdio** para conexiones locales
* **Streamable HTTP** para conexiones remotas

Cuando desarrollas un servidor, el protocolo recomienda soportar **stdio** siempre que sea posible. Soportar stdio facilita a los usuarios conectarse a tu servidor cargando el script del servidor localmente en su aplicación y permitiendo que el cliente MCP lo invoque mediante un comando establecido (usualmente documentado en el README del servidor).

Al usar este transporte, el cliente MCP ejecuta el script del servidor como un **subproceso de larga duración**, y los mensajes se leen desde la **entrada estándar** del servidor y se escriben en su **salida estándar**.

**Figura 5-1** muestra el flujo básico de una conexión stdio.

---

Por otro lado, **Streamable HTTP** está diseñado para soportar conexiones remotas, de manera que el servidor y el cliente **no necesitan estar en el mismo sistema**.

Al implementar un servidor con este transporte, debes proporcionar un único **endpoint HTTP** (llamado MCP endpoint en la especificación) que soporte **GET y POST**, ya que Streamable HTTP utiliza ambos métodos:

* **GET**: para que el cliente inicie la conexión con el servidor
* **POST**: para que el cliente envíe solicitudes, notificaciones y eventos al servidor

Lo que hace que Streamable HTTP sea “streamable” es que permite opcionalmente **SSE streaming** entre el cliente y el servidor. Esto permite, entre otras cosas, que una sola instancia de cliente MCP se conecte a **múltiples servidores simultáneamente** usando flujos separados.

**NOTA:** Aunque Streamable HTTP está pensado principalmente para servidores remotos, también puede usarse localmente, lo que ayuda en depuración y pruebas. Al usar este transporte localmente, la especificación recomienda **localhost (127.0.0.1)** en lugar de 0.0.0.0.

**Figura 5-2** muestra el flujo básico de una conexión Streamable HTTP.

---

### Sesiones remotas y autorización

Debido a los desafíos únicos de ejecutar un servidor remoto disponible en Internet público, la especificación MCP contempla:

* **Reanudación de sesiones de streaming**
* **Autorización vía OAuth**

Para soportar la reanudación de sesiones, tu servidor debe incluir un campo **id** en cada evento SSE con un **ID globalmente único** para ese evento. Si la conexión de streaming se interrumpe, el cliente puede enviar una solicitud **GET** con el encabezado `_Last-Event-ID_` al servidor con el último ID de evento recibido. El servidor puede reenviar los mensajes que se hubieran enviado después de ese ID en el mismo flujo. (Como el servidor puede tener múltiples flujos abiertos a la vez, nunca debes enviar mensajes destinados a flujos diferentes).

> **“Don’t cross the streams”**
> Dr. Egon Spengler, Ghostbusters (1984)

Aunque la reanudación de sesiones de streaming es opcional, la **autorización es obligatoria** al ejecutar un servidor remoto. La especificación MCP permite autorización mediante **OAuth**, y generalmente requiere que el desarrollador del servidor use una **biblioteca OAuth nativa del lenguaje** para manejar el flujo de autorización.

La naturaleza de MCP y la IA generativa en general expone a los servidores MCP a amenazas conocidas y nuevas, como ataques **man-in-the-middle**, secuestro de sesiones, ataques de token passthrough, envenenamiento de herramientas, entre otros. Defenderse contra estos riesgos debe ser prioritario para proteger los datos propios y los de los usuarios. Este tema se tratará con mayor detalle en **Seguridad del Servidor**, más adelante en este capítulo.

---

### Gestión de conexiones

Fuera de la seguridad y autorización, la mayoría de los detalles de **gestión de conexiones** son abstraídos por los SDKs del protocolo MCP, de manera similar a lo que viste en los ejemplos de cliente del Capítulo 3.

En la siguiente sección se introducirán **las dos formas de construir servidores MCP con el SDK de Python**, y luego se profundizará en cómo usar una de estas técnicas en tus propios proyectos.

---


---

## Construyendo Servidores con el SDK de Python

Cuando MCP se lanzó por primera vez con sus diversos SDKs, solo existía **una forma de construir servidores MCP**. Esto normalmente no sorprende: los SDKs suelen ser opinativos sobre cómo se construye con ellos, y este no fue la excepción. Sin embargo, poco después del lanzamiento de MCP, se publicó **FastMCP**, que proporcionó una manera mucho más simplificada de construir servidores MCP en Python. Resultó ser tan popular que se incorporó oficialmente al **SDK oficial de Python de MCP**, lo que dio lugar a una estructura de **2 niveles para el desarrollo de servidores MCP**:

* **FastMCP**: para casi todos los casos de uso
* La API “**low-level**” preexistente: para quienes necesitan más control

---

### ADVERTENCIA

No confundas **FastMCP 1.0** con **FastMCP 2.0** (o FastAPI).

* FastMCP 1.0 se centró en **rediseñar la experiencia de desarrollo de servidores MCP**.
* FastMCP 2.0 amplía su enfoque a **todo el SDK de Python**, implementando MCP con Python usando su propio diseño.

---

### ANTIPATRÓN: CREAR SERVIDORES MCP A PARTIR DE REST APIs

Un error común en comunidades MCP y servidores lanzados es la **generación automática de servidores MCP desde APIs REST existentes**. Esto es un **antipatrón**, exacerbado por productos y herramientas que prometen hacer exactamente eso para organizaciones que quieren exponer sus APIs a agentes.

Las APIs REST suelen ser **muy granulares, sin estado y polimórficas**, mientras que los agentes funcionan mejor cuando una herramienta realiza **una acción única y bien definida** con una estructura igualmente clara de inputs y outputs.

Si necesitas llamar a **varios endpoints de una API REST** para realizar una sola actividad, enfrentas dos riesgos:

1. **Aumento de errores en la elección de herramientas**
2. **Incremento exponencial del costo de las acciones del agente**

Por ejemplo, si tu agente tiene un **95% de éxito** al elegir la herramienta correcta para una acción, tener que elegir 5 herramientas reduce la tasa de éxito a aproximadamente **77%**.

Esto también explica el **incremento de costos**: si el agente llama a 5 herramientas para hacer una sola acción, los tokens de entrada de cada turno incluyen los tokens de entrada y salida de los turnos anteriores, aumentando rápidamente. Usando un cálculo aproximado basado en el ejemplo del Capítulo 3, el uso total de tokens podría **incrementar 8x** y el costo **casi 7x**.

**Recomendación**: en lugar de desplegar un servidor MCP autogenerado, sigue el consejo de **Jeremiah Lowin, CEO de Prefect**:

1. Usa un servidor MCP autogenerado para **bootstrap** de tu servidor final.
2. **Curate agresivamente** las herramientas generadas, eliminando las innecesarias, variables ambiguas y más.
3. Mejor aún: construye tu servidor **desde cero**, usando **“agent stories”** al estilo de los user stories en desarrollo de software enfocado en productos, para crear un servidor que el agente pueda usar de forma óptima.

---

### FastMCP

**FastMCP** es ahora la forma **predeterminada** de construir servidores MCP con el SDK de Python. Proporciona gran parte de la potencia del diseño original del SDK con **una interfaz mucho más simple**.

* Interactúa con tu código de servidor mediante **decoradores**, similar a FastAPI
* Soporta nativamente funciones como **autenticación del servidor**
* Soporta todas las **primitivas MCP** y capacidades del lado del cliente, como **elicitations** y **sampling**
* Permite gestionar el **ciclo de vida del servidor**, por ejemplo: abrir una conexión a la base de datos al instanciar el servidor y cerrarla al detenerlo

A nivel básico, solo necesitas **instanciar el objeto FastMCP** y pasarle el nombre de tu servidor:

```python
# Ejemplo 5-1. Instanciando un servidor FastMCP
from mcp.server.fastmcp import Context, FastMCP

mcp = FastMCP(name="server_name")
```

Esto simplemente instancia el objeto FastMCP, que luego usarás para **agregar primitivas MCP** a tu servidor. El constructor de FastMCP permite muchas otras opciones, como **instrucciones mostradas al cliente**, **modo debug**, **nivel de log predeterminado**, **flag de respuesta JSON**, entre otros. Todas estas opciones se exponen como un **objeto Context**, que puede pasarse a cualquier función del servidor para su uso dentro de ellas.

Luego, puedes iniciar el servidor con:

```python
# Ejemplo 5-2. Ejecutando un servidor FastMCP
if __name__ == "__main__":
    mcp.run()
```

Esto bloqueará la ejecución hasta que el servidor se detenga. También puedes pasar un **método de transporte** al método `run()`, que iniciará el servidor y retornará una **coroutine** que puedes `await` para arrancarlo.

Además de la ejecución directa, puedes usar [Link to Come] para:

* Ejecutar el servidor en **modo desarrollo**: `uv run mcp dev server.py`
* Instalarlo para correr mediante **Claude Desktop**: `uv run mcp install server.py`

Estas técnicas funcionan **solo con servidores FastMCP**, no con servidores creados con la API low-level, y requieren que tu proyecto tenga instaladas las **herramientas de desarrollo MCP**, que puedes instalar con:

```bash
uv add "mcp[cli]"
```


Aquí tienes la traducción con formato, respetando las ideas originales:

---

## Low-Level

La **API low-level** proporciona el **protocolo MCP completo**, incluyendo todas las funciones de la API FastMCP, pero también te da **control total** sobre cómo manejar las solicitudes del cliente, el ciclo de vida del servidor y cómo manejar las llamadas a herramientas.

En FastMCP, simplemente decoras funciones de Python con algo como `@mcp.tool()` (verás exactamente cómo en **“Primitivas del Servidor: Tools, Prompts y Resources”**) y cuando un cliente solicita la lista de herramientas de tu servidor, **FastMCP se encarga de construir y devolver la lista automáticamente**.

Con la API low-level, puedes **implementar tus propias respuestas** para la solicitud `tool/list`, dándote más control sobre lo que tu servidor devuelve al cliente.

Para instanciar un servidor con la API low-level, debes:

1. Importar `Server` desde `mcp.server.lowlevel`
2. Crear una función **async**
3. Configurar las **opciones de inicialización** del servidor (nombre, versión, capacidades, etc.)
4. Instanciar el servidor dentro de un **context manager `async stdio_server()`**

Al instanciar el servidor, también debes pasar:

* Un **read stream** y un **write stream**, obtenidos del context manager
* Las **opciones de inicialización**

Luego, simplemente llamas a esa función con `asyncio.run()` para iniciar el servidor de manera **asíncrona**.

---

### Ejemplo 5-3. Iniciando un servidor MCP low-level

```python
import asyncio
import mcp.server.stdio
from mcp.server.lowlevel import NotificationOptions, Server
from mcp.server.models import InitializationOptions

# Crear una instancia del servidor
server = Server("low-level-server")

async def run():
    print("Running low-level server")
    
    initialization_options = InitializationOptions(
        server_name="low-level-server",
        server_version="0.1.0",
        capabilities=server.get_capabilities(
            notification_options=NotificationOptions(),
            experimental_capabilities={},
        ),
    )

    async with mcp.server.stdio.stdio_server() as (read_stream, write_stream):
        await server.run(
            read_stream=read_stream,
            write_stream=write_stream,
            initialization_options=initialization_options,
        )

if __name__ == "__main__":
    asyncio.run(run())
```



Como puedes ver, **instanciar un servidor con la API low-level es más verboso** que con FastMCP, pero te da **capacidades y flexibilidad adicionales**, aunque rara vez sean necesarias.

Un aspecto que no se explicó en la introducción de este ejemplo es el método **`get_capabilities()`**. Este es interesante porque **verifica si el servidor tiene algún handler registrado** para una capacidad, como soporte de **prompts, resources o tools**.

* Si se encuentran esos handlers, se crea una instancia de la clase que representa esa capacidad (por ejemplo, `PromptsCapability`) y se pasa a un objeto `ServerCapabilities`, que luego se incluye en el objeto **`InitializationOptions`**.
* Así es como se maneja la **declaración de capacidades**, algo obligatorio según la especificación MCP, en la API low-level.
* El objeto `NotificationOptions` pasado a `get_capabilities()` se usa para determinar **qué capacidades soportan notificaciones**.

---

### Implementación de herramientas (tools)

Para soportar el uso de herramientas, debes **implementar un handler** para las solicitudes:

* `tools/list`
* `tool/call`

Lo mismo aplica para **prompts** y **resources**.

Como verás en **“Primitivas del Servidor: Tools, Prompts y Resources”**, este proceso es **más detallado que en FastMCP**, que infiere tu lista de herramientas a partir de las funciones declaradas como tools.

Para manejar la solicitud de listar herramientas, debes escribir una función que **devuelva una lista de objetos Tool**.

Cada instancia de **Tool** tiene:

* `name` (nombre)
* `description` (descripción)
* `inputSchema` (esquema de entrada)
* Opcionalmente, `outputSchema` (esquema de salida)

El **inputSchema** debe ser un **objeto JSON** con:

* `type`
* `properties` → un diccionario donde las claves son los argumentos de la función de la herramienta y los valores son diccionarios con el **tipo** y la **descripción** del argumento
* `required` → lista de argumentos obligatorios

En el ejemplo siguiente, esto se implementa como `list_tools()`. La herramienta se implementa en `add()` y se declara con el **decorador `@server.call_tool()`**.

Todas las funciones de tools en la API low-level reciben:

* **name** → nombre de la herramienta
* **args** → diccionario con los argumentos de la herramienta y sus valores

---

### Lógica de la herramienta

* Para asegurarte de que se llama a la herramienta correcta, puedes revisar el contenido del **parámetro `name`**.
* Para usar los argumentos definidos en el `inputSchema`, accedes a ellos como **claves en el diccionario `args`**.
* Esto te proporciona todos los elementos necesarios para implementar la **lógica de la herramienta**.

Si la función retorna algo, puede devolverse como:

* Un objeto **Content** (por ejemplo, `TextContent`)
* O un diccionario que **declare el tipo del resultado** junto con el resultado mismo

En el ejemplo siguiente, se usa este último enfoque, devolviendo un tipo `text` con un **f-string** que muestra el cálculo y resultado de la herramienta.



### Ejemplo 5-4. Listando herramientas y haciéndolas disponibles mediante un servidor MCP low-level

```python
from typing import Any
import mcp.server.stdio
...
from mcp.types import Tool
...

@server.list_tools()
async def list_tools() -> list[Tool]:
    """Lista todas las herramientas disponibles en el servidor."""
    return [
        Tool(
            name="add",
            description="Suma dos números.",
            inputSchema={
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "El primer número a sumar"
                    },
                    "b": {
                        "type": "number",
                        "description": "El segundo número a sumar"
                    },
                },
                "required": ["a", "b"],
            },
        )
    ]


@server.call_tool()
async def add(name: str, args: dict[str, Any]) -> dict[str, Any]:
    """Suma dos números.

    Args:
        a: Primer número
        b: Segundo número
    """
    if name != "add":
        raise ValueError(f"Herramienta desconocida: {name}")
    result = args["a"] + args["b"]
    return {"type": "text", "text": f"{args['a']} + {args['b']} = {result}"}
```

Este ejemplo sencillo ilustra **el proceso básico** de implementar una función que lista primitivas (como tools) y la lógica de una herramienta.

Además, puedes proporcionar un **output schema estructurado**, que asegura que la salida de la herramienta se entregue en un formato específico al cliente MCP y, por ende, a la aplicación host.

* El **output schema** permite que la herramienta valide su salida antes de enviarla al cliente.
* La herramienta puede devolver **solo datos estructurados** o una **tupla de datos estructurados y bloques de contenido legibles**, en lugar de enviar solo bloques de contenido como en el ejemplo anterior.
* En **FastMCP**, esto se genera automáticamente a partir de la **anotación de tipo del retorno de la función**.

---

### Ejemplo 5-5. Retornando datos estructurados desde una herramienta con output schema

```python
@server.list_tools()
async def list_tools() -> list[Tool]:
    """Lista todas las herramientas disponibles en el servidor."""
    return [
        Tool(
            name="add",
            description="Suma dos números.",
            inputSchema={
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "El primer número a sumar"},
                    "b": {"type": "number", "description": "El segundo número a sumar"},
                },
                "required": ["a", "b"],
            },
            outputSchema={
                "type": "object",
                "properties": {
                    "augend": {"type": "number", "description": "El primer número a sumar"},
                    "addend": {"type": "number", "description": "El segundo número a sumar"},
                    "sum": {"type": "number", "description": "El resultado de la suma"},
                },
                "required": ["augend", "addend", "sum"],
            },
        )
    ]

@server.call_tool()
async def add(name: str, args: dict[str, Any]) -> dict[str, Any]:
    """Suma dos números.

    Args:
        a: Primer número
        b: Segundo número
    """
    if name != "add":
        raise ValueError(f"Herramienta desconocida: {name}")
    result = {"augend": args["a"], "addend": args["b"], "sum": args["a"] + args["b"]}
    return result
```

---

### Uso del output schema en la API low-level

Para usar un **output schema** se requieren dos pasos:

1. **Declarar el schema** en la definición correcta de la herramienta dentro del handler `list_tools()`.
2. **Retornar los datos estructurados** desde la propia herramienta.

* La definición del schema de salida tiene la **misma estructura que el input schema**: un diccionario Python que imita un objeto JSON, con:

  * `type: "object"`
  * `properties`: los componentes esperados de la salida
  * Opcionalmente, `required`: lista de propiedades obligatorias

* Si no se define ninguna como obligatoria, cualquier salida será válida.

* En la herramienta, asegúrate de devolver un diccionario con las **mismas claves que las definidas en `properties` del output schema**.

MCP validará automáticamente la salida contra el schema y generará un **error si faltan campos obligatorios, están mal escritos, tienen el tipo incorrecto, etc.**

---


Como puedes ver, **intentar cambiar el nombre de uno de los campos en el resultado de `add()`** y ejecutar tu servidor y herramienta en **MCP Inspector** mostrará cómo la validación del **output schema** funciona automáticamente.

El servidor **low-level** requiere mucho más trabajo solo para un ejemplo simple como este, pero tiene beneficios, como poder trabajar con la **API de lifespan del servidor**.

---

### API de Lifespan

La **lifespan API** permite algunas cosas, principalmente:

* Configurar y cerrar cualquier recurso que el servidor necesite (por ejemplo, conexiones a bases de datos) al **iniciar** y **detener** el servidor.
* Esto es útil para recursos **costosos de crear** o que requieren tiempo para establecer la conexión, y para aquellos que necesitan cerrarse explícitamente al detener el servidor.

Para usarla:

1. Define una función **async** decorada con `@asynccontextmanager` de `contextlib`.
2. La función debe recibir un argumento de tipo **Server** y devolver un `AsyncIterator`.
3. Dentro de la función:

   * En el **cuerpo principal (try)**, configura los recursos y haz `yield` de los objetos que quieres pasar al contexto como diccionario.
   * En el **finally**, haz el teardown de los recursos.
4. Pasa esta función al constructor de `Server` usando el argumento **lifespan**.

---

### Acceso al contexto de lifespan

Puedes proporcionar estos recursos a las herramientas, prompts y resources de tu servidor mediante un **context object**.

* Para acceder al contexto dentro de una función que enfrenta al agente, usa la propiedad `request_context` del servidor.
* Esto te dará el diccionario que devolviste desde la función lifespan.

---

### Ejemplo 5-6. Usando la API low-level para gestionar el lifespan del servidor

```python
import asyncio
import sys
from collections.abc import AsyncGenerator
from contextlib import asynccontextmanager
from datetime import datetime

logs = []

@asynccontextmanager
async def lifespan(server: Server) -> AsyncGenerator[list[str]]:
    logs.append(f"{datetime.now()}: Server started")
    try:
        logs.append(f"{datetime.now()}: logs retrieved")
        yield logs
    finally:
        print(logs, file=sys.stderr)

# Crear instancia del servidor con lifespan
server = Server("low-level-server", lifespan=lifespan)

@server.list_tools()
async def list_tools() -> list[Tool]:
    """Lista todas las herramientas disponibles en el servidor."""
    ctx = server.request_context
    print(ctx.lifespan_context[-1], file=sys.stderr)
    print(logs[-1], file=sys.stderr)
    return [
        Tool(
            name="add",
            description="Suma dos números.",
            inputSchema={
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "El primer número a sumar"},
                    "b": {"type": "number", "description": "El segundo número a sumar"},
                },
                "required": ["a", "b"],
            },
            outputSchema={
                "type": "object",
                "properties": {
                    "augend": {"type": "number", "description": "El primer número a sumar"},
                    "addend": {"type": "number", "description": "El segundo número a sumar"},
                    "sum": {"type": "number", "description": "El resultado de la suma"},
                },
                "required": ["augend", "addend", "sum"],
            },
        )
    ]

@server.call_tool()
async def add(name: str, args: dict[str, Any]) -> dict[str, Any]:
    """Suma dos números."""
    if name != "add":
        raise ValueError(f"Herramienta desconocida: {name}")
    result = {"augend": args["a"], "addend": args["b"], "sum": args["a"] + args["b"]}
    ctx = server.request_context
    logs = ctx.lifespan_context[-1]
    print(logs, file=sys.stderr)
    return result

async def run():
    print("Running low-level server", file=sys.stderr)
    print(logs, file=sys.stderr)
```

---

### Qué hace este ejemplo

1. **Logging rudimentario**:

   * Se crea una lista global `logs` para almacenar los mensajes de log.
   * La función `lifespan()` agrega mensajes cuando el servidor se inicia y cuando los logs se recuperan, luego hace `yield logs`.
   * En el `finally`, se imprimen los logs al cerrar el servidor.

2. **Inicialización del servidor**:

   * Se pasa la función `lifespan()` al parámetro `lifespan` del constructor `Server`.

3. **Acceso al contexto**:

   * Dentro de `list_tools()` y `add()`, se accede a `server.request_context` para obtener el contexto de lifespan y actualizar los logs.
   * Se imprime el último mensaje en `stderr`, lo que permite **ver los logs en MCP Inspector** en el panel de notificaciones.

---

## ¿Por qué FastMCP?

Mientras que la API de lifespan es indudablemente genial, tiene aplicabilidad limitada, lo que hace difícil justificar renunciar a la simplicidad de FastMCP. Por eso, en este libro nos enfocaremos en construir servidores con FastMCP en lugar de la API de bajo nivel. Pero más allá (y gracias a) la combinación de potencia y simplicidad de FastMCP, rápidamente se ha convertido en la forma predeterminada de construir servidores MCP con el SDK de Python. Debido a su ubicuidad, todo el resto del texto y los ejemplos de este capítulo se centrarán en la API de FastMCP.

---

**Primitivos del Servidor: Herramientas, Prompts y Recursos**
Cuando desarrollas un servidor MCP, la mayor parte de tu enfoque probablemente estará en implementar los primitivos que tu servidor va a soportar. Los primitivos de MCP son los principales bloques de construcción que un servidor MCP proporciona a los clientes conectados y a las aplicaciones host. Estos primitivos son:

* **Herramientas (tools)**: funciones que la aplicación host puede elegir llamar.
* **Prompts (indicaciones)**: contenido que puedes proporcionar a la aplicación host para guiar mejor cómo interactúa con lo que el servidor ofrece.
* **Recursos (resources)**: datos que el servidor proporciona a la aplicación host.

En esta sección aprenderás a servir cada uno de estos primitivos desde un ejemplo de servidor MCP. En el Capítulo 3 usaste un servidor para ejecutar los ejemplos del cliente, y ahora en el Capítulo 54 verás cómo se armó ese servidor.

---

**Primero lo primero: cómo crear y arrancar un servidor MCP**

```python
from mcp.server.fastmcp import FastMCP

# Inicializar servidor FastMCP
mcp = FastMCP("minimal-stdio-server")

if __name__ == "__main__":
    # Inicializar y ejecutar el servidor
    mcp.run()
```

Este ejemplo es justo lo que viste en la sección de FastMCP, y crea una instancia de un servidor FastMCP con el nombre `minimal-stdio-server`, y lo ejecuta con `mcp.run()` cuando el script se ejecuta directamente.

Puedes ejecutarlo con:

```bash
uv run server.py
```

o con:

```bash
uv run mcp dev server.py
```

Esto iniciará el servidor y no hará nada más. En adelante, todos los ejemplos podrán ejecutarse de esta manera, y se recomienda encarecidamente usar MCP Inspector para interactuar fácilmente con el servidor en los ejemplos.

---

**TIP**
Cuando uses MCP Inspector, puedes usar *flags* de línea de comando para instalar dependencias adicionales o montar código local para ver tus cambios en tiempo real en la interfaz de MCP Inspector:

* Para dar acceso a tu servidor a la librería Pydantic sin instalarla en tu entorno local:

```bash
uv run mcp dev server.py --with pydantic
```

* Para montar código local y poder hacer ediciones sin reiniciar el servidor:

```bash
uv run mcp dev server.py --with-editable .
```

---

**Soporte para Streamable HTTP**

Si quieres que tu servidor soporte Streamable HTTP, puedes pasar el string de transporte apropiado (`'streamable-http'`) al método `run()`, que puedes hacer configurable de varias formas si lo necesitas:

```python
if __name__ == "__main__":
    # Inicializar y ejecutar el servidor
    mcp.run("streamable-http")
```

Y esto es todo lo que se necesita para construir un servidor MCP en funcionamiento. Por supuesto, estos ejemplos no hacen nada más que ejecutarse, y en la siguiente sección se cambiará eso implementando una herramienta que pueda ser llamada por un cliente MCP.


**Sirviendo Herramientas**
Las herramientas son la columna vertebral del ecosistema MCP. Son lo que realmente amplía el comportamiento y las capacidades de un LLM, al proporcionarle acceso a prácticamente cualquier cosa que puedas programar, dándole instantáneamente la capacidad de hacer cualquier cosa que puedas imaginar.

Pero, ¿qué es exactamente una herramienta? En el SDK de Python, una herramienta es simplemente una función de Python decorada con `@mcp.tool()`. Esto permite que FastMCP envíe automáticamente el nombre y la descripción de la herramienta al cliente cuando solicita la lista de herramientas del servidor, la cual normalmente se pasaría al modelo de lenguaje de la aplicación host. Con FastMCP, los esquemas de entrada y salida de la herramienta se infieren automáticamente a partir de la firma de la función y cualquier anotación de tipo que proporciones, mientras que la descripción de la herramienta se infiere del docstring de la función de la herramienta.

> MCP es como Matrix, te conectas a un servidor y de repente sabes kung fu.
> — Christopher Bailey, anfitrión del Real Python Podcast

---

**Diseñando Herramientas**
Antes de ver ejemplos, es importante entender cómo diseñar una herramienta para un uso efectivo por parte del LLM. Ya hablamos de la importancia de usar la API de FastMCP para que el nombre de la herramienta, los parámetros y la descripción puedan ser detectados por FastMCP y colocados en las partes adecuadas del modelo de datos. Pero no solo deben ser detectados por FastMCP, también deben ser entendidos por el LLM.

Esto significa que las herramientas deben:

* Tener nombres distintivos que se relacionen con las acciones que realizan.
* Descripciones claras, pero no demasiado largas.
* Entradas y salidas bien definidas.

También se recomienda mantener tus herramientas dentro de **namespaces** apropiados. Con MCP, un namespace normalmente se muestra como un prefijo separado por guiones bajos en el nombre de la herramienta y es útil si desarrollas múltiples herramientas similares con una diferencia clave (como un proveedor de API de terceros). Incluso si no es el caso, los namespaces ayudan al usuario, especialmente si un agente definido por un propósito utiliza varias herramientas similares.

Anthropic también recomienda descripciones de herramienta muy detalladas, comparando su redacción con explicar cómo funciona un código a un nuevo integrante del equipo. Debes pensar en el contexto que posees implícitamente sobre ese código y hacerlo explícito. Lo mismo aplica para un LLM: se necesita balancear eficiencia de tokens con claridad y contexto que permita un rendimiento óptimo al elegir herramientas.

---

Con tu código de herramienta bien documentado, debes enfocarte en lo que la herramienta hace. Los LLMs funcionan mejor con herramientas que sean **acciones de extremo a extremo**, en lugar de acciones individuales y granulares típicas de APIs REST. Esto es porque la aplicación host necesitará llamar varias herramientas para lograr una sola acción, lo que puede aumentar los errores y el costo en tokens, y saturar la ventana de contexto. Puedes dividir el código de la herramienta en funciones privadas y componerlas dentro del servidor para mejorar la testabilidad y reutilización, solo cuida de no exponer esos bloques como herramientas.

---

En desarrollo de software tradicional, muchas estrategias de gestión de producto incluyen crear **“user stories”**, descripciones cortas de una función o acción desde la perspectiva del usuario, como:

> “Como [tipo de usuario], quiero hacer [acción] para poder [objetivo]”.

En MCP server development, como construyes herramientas para un agente, puede ser útil crear **“agent stories”**, describiendo acciones que un agente podría querer realizar desde la perspectiva del agente.

---

Además, ten en cuenta que los LLMs tienen límites suaves sobre cuántas herramientas pueden manejar antes de que la precisión en su elección disminuya. Como desarrollador de servidor, intenta mantener un número razonable de herramientas expuestas. Los usuarios apreciarán tener unas pocas herramientas bien definidas y confiables.

**NOTA**
Los desarrolladores de clientes tienen más opciones para lidiar con demasiadas herramientas disponibles, como filtrar la lista de herramientas que reciben (a costa de flexibilidad) o desarrollar sistemas multiagente con subagentes especializados, en comparación con usuarios finales que usan tu servidor directamente en una aplicación.

---

Como se mencionó antes, la API de FastMCP infiere automáticamente los esquemas de entrada y salida de la firma de la función de la herramienta. Una consecuencia de esto es que muchas herramientas MCP devuelven **salida estructurada validada por defecto**, asegurando que la salida tenga una forma consistente y sea más fácil de manejar. Estos tipos de salida incluyen:

* Modelos Pydantic
* TypedDicts de Python
* Dataclasses
* Clases con type hints
* Diccionarios con claves string y valores serializables en JSON
* Tipos primitivos de Python (envueltos en un diccionario con la clave `result`)
* Tipos genéricos de Python (envueltos en un diccionario con la clave `result`)

Aunque esto cubre casi todos los tipos de salida que puedes retornar en Python, la lista más actualizada está en el README del SDK de Python.

Adicionalmente, FastMCP proporciona tipos **Image** y **Audio** que pueden usarse como salidas de herramientas, construyendo automáticamente bloques de respuesta `ImageContent` y `AudioContent` para el cliente, sin trabajo adicional por tu parte ni del desarrollador del cliente.

El siguiente ejemplo muestra herramientas que devuelven un modelo Pydantic y una imagen.


**Ejemplo 5-8. Herramientas que devuelven un modelo Pydantic y una imagen**

```python
from mcp.server.fastmcp import FastMCP, Image
from PIL import Image as PILImage
from PIL import ImageDraw
from pydantic import BaseModel

# Inicializar servidor FastMCP
mcp = FastMCP("structured-output-server")


class ReportCard(BaseModel):
    name: str
    grades: list[tuple[str, int]]  # nombre de la clase y calificación


@mcp.tool()
async def generate_report_card(name: str, grades: list[tuple[str, int]]) -> ReportCard:
    """
    Genera una boleta de calificaciones para un estudiante.

    Args:
        name: El nombre del estudiante
        grades: Lista de tuplas con el nombre de la clase y la calificación
    """
    return ReportCard(name=name, grades=grades)


@mcp.tool()
async def generate_report_card_image(report_card: ReportCard) -> Image:
    """
    Genera una imagen de la boleta de calificaciones de un estudiante.

    Args:
        report_card: La boleta de calificaciones para generar la imagen
    """
    image = PILImage.new("RGB", (400, 200), color=(255, 255, 255))
    draw = ImageDraw.Draw(image)
    draw.text((100, 100), report_card.name, fill=(0, 0, 0))
    return Image(data=image.tobytes())


if __name__ == "__main__":
    # Inicializar y ejecutar el servidor
    mcp.run()
```

En este ejemplo, tenemos un modelo Pydantic llamado `ReportCard` que representa la boleta de calificaciones de un estudiante e incluye su nombre y una lista de tuplas de clase-calificación. Esto define el formato de salida estructurado para la herramienta y la función `generate_report_card()`.

La implementación de la herramienta `generate_report_card()` toma los datos de entrada y los devuelve en el formato estructurado `ReportCard`.

La otra herramienta, `generate_report_card_image()`, muestra cómo funciona la clase `Image` de FastMCP: se importa la clase `Image` de FastMCP y se devuelve un objeto de imagen construido desde esa clase. En este ejemplo, usamos Pillow para crear una imagen, dibujar sobre ella y agregar el nombre del estudiante. Debemos devolver un objeto `fastMCP.Image`, por lo que convertimos la imagen a bytes usando el método `tobytes()` y lo pasamos al constructor de `Image`, devolviendo el objeto `Image` instanciado. No es obligatorio usar Pillow para manipular la imagen, pero sí se necesita poder convertirla a bytes.

Los desarrolladores de servidores tienen responsabilidades importantes de seguridad respecto a las herramientas. Como las herramientas pueden ejecutarse en la máquina de un usuario, en un servidor de la aplicación o en un servidor controlado por ti, debes controlar los inputs que reciben, qué outputs generan y quién puede acceder a ellos. Esto implica validar todos los inputs (tipo y contenido), sanear cualquier output sensible, implementar controles de acceso cuando sea necesario (por ejemplo, acceso autenticado a un servicio de terceros) y limitar la tasa de llamadas a las herramientas. Estas acciones protegen contra abusos y salvaguardan datos sensibles, dependiendo de lo que haga cada herramienta. Construir medidas de seguridad efectivas también requiere entender cómo se usan las herramientas dentro del protocolo y el orden de operaciones en las interacciones modelo-cliente-servidor.

---

**Cómo se usan las herramientas**

Generalmente, las herramientas son usadas por el modelo cuando y cómo este decide llamarlas. Esto es un componente necesario de la agencia según Anthropic. Las herramientas no están limitadas a este modelo de interacción: los desarrolladores de aplicaciones pueden implementar cualquier modelo que funcione para su aplicación, como un flujo agentic que llame herramientas de manera controlada o un modelo híbrido donde el LLM elige algunas herramientas y la aplicación maneja otras partes del flujo. Esta elección, junto con seguir las directrices de la especificación MCP sobre interacción con herramientas, depende del desarrollador de la aplicación, mientras que las responsabilidades del desarrollador del servidor son proveer la lista de herramientas, las herramientas mismas y, según la estrategia de despliegue, el entorno de ejecución de las herramientas.

Cuando una aplicación se conecta a un servidor mediante un cliente, este primero envía una solicitud de lista de herramientas al servidor. En el Python SDK, esto llama a la función `list_tools()` del servidor, implementada automáticamente por FastMCP o manualmente usando la API de bajo nivel. Esto devuelve una lista de herramientas al cliente, que se envía con cada solicitud posterior al modelo cuando actúa como agente. El modelo selecciona qué herramienta usar, si alguna, y si se selecciona, el cliente envía una solicitud `tool call` al servidor. En el Python SDK, esto llama a la función `call_tool()` del servidor o a la función `tool()` de FastMCP, que decora las llamadas y permite invocar la función correcta con los argumentos apropiados. Esto valida los argumentos de entrada y salida contra el esquema de la herramienta y devuelve los resultados al cliente.

---

**UN REPASO SOBRE DECORADORES**

Los decoradores son un poco de “azúcar sintáctica” en Python que permiten modificar el comportamiento de una función sin cambiar el código de la función misma. Funcionan porque las funciones en Python son objetos de primera clase, lo que significa, entre otras cosas, que puedes pasar funciones como argumentos a otras funciones.

Cuando decoras una función, la función decorada se pasa como argumento a la función decoradora, la cual llamará a la función decorada en algún momento durante la ejecución de su lógica. Esto se hace definiendo una función `wrapper()` o `handler()` dentro del cuerpo de la función decoradora (el nombre de la función interna no importa; son solo convenciones comunes) que ejecuta su lógica y llama a la función que recibió. Esa función interna es devuelta por la función externa (decoradora).

En la API de bajo nivel del SDK Python de MCP, esta es la función `call_tool()`. Tiene dos funciones internas anidadas. La más interna (`handler()`) maneja la solicitud, analiza el nombre de la herramienta y los argumentos, y obtiene la definición de la herramienta desde la caché interna del servidor. Opcionalmente valida los argumentos contra el esquema de entrada de la herramienta, luego llama a la función de la herramienta con su nombre y argumentos. Toma los resultados y los analiza en salidas estructuradas o no estructuradas según corresponda, luego valida la salida contra el esquema de salida de la herramienta. Después crea un objeto `ServerResult` del SDK y lo retorna.

La función que envuelve a esta se llama `decorator()`, que recibe la función como argumento, define la función interna `handler` y la registra agregándola al diccionario `request_handlers` del servidor. La función original es devuelta, y la función externa retorna la función decoradora, como es típico en los decoradores. Este flujo suele ser confuso, y se complica aún más por la complejidad inherente de `call_tool()`.

El decorador `tool()` de la API FastMCP es un poco más simple. Solo tiene una función interna llamada `decorator()`, que toma una función como argumento y llama a `add_tool()` en la instancia del servidor, agregando la definición de la herramienta (como un objeto `Tool`) a la caché interna del servidor, implementada en una clase separada llamada `ToolManager`. Si se necesita llamar a la herramienta, se invoca el método interno `call_tool()` del `ToolManager`, que obtiene la herramienta del registro y llama a su método `run()`. Este método, como el `handler()` de la API de bajo nivel, valida opcionalmente los argumentos y la salida y convierte el resultado al tipo de salida apropiado.

También se pueden pasar parámetros opcionales al decorador `tool()`, como:

* `name`: sobreescribe el nombre de la función de la herramienta.
* `title`: proporciona un nombre legible para humanos.
* `description`: sobreescribe el docstring de la función.
* `annotations`: recibe un objeto `ToolAnnotations` con propiedades de ayuda para describir mejor la herramienta.
* `structured_output`: si se establece en `True`, la herramienta siempre devuelve salida estructurada; si es `False`, nunca; si no se define, intenta inferir el tipo de salida desde la anotación de retorno de la función.

Los decoradores suelen ser difíciles de entender, y al escribir los propios a menudo es útil consultar tutoriales. Un recurso recomendado es *Primer on Python Decorators* de Geir Arne Hjelle en Real Python.

El resultado de la herramienta se pasa al cliente, que lo enviará al modelo para continuar su bucle de agente. Además, el servidor puede enviar notificaciones `list_changed` al cliente cada vez que cambia la lista de herramientas disponibles. En el SDK de Python, esto se puede enviar manualmente con `ctx.session.send_tool_list_changed()`, donde `ctx` es el objeto de contexto MCP que se puede pasar como parámetro a una función de herramienta. Dependiendo de cómo esté implementado el cliente, esto puede hacer que el cliente realice una solicitud de lista de herramientas de vuelta al servidor, actualizando la lista del cliente.

A continuación, se muestra un ejemplo que implementa una herramienta con espacio de nombres (`namespace`) que devuelve salida estructurada mediante un modelo Pydantic y establece un nombre más legible para la herramienta. También crea una herramienta más simple que devuelve salida no estructurada, forzando al servidor a devolver siempre salida no estructurada y agregando una anotación de solo lectura a la herramienta.


**Ejemplo 5-9. Implementación completa de una herramienta usando las funcionalidades de la API FastMCP**

```python
from random import randint

from mcp.server.fastmcp import FastMCP
from mcp.types import ToolAnnotations
from pydantic import BaseModel

# Inicializar servidor FastMCP
mcp = FastMCP("full-tool-server")


class Class(BaseModel):
    title: str
    grade: int
    instructor: str
    credits: int


class ReportCard(BaseModel):
    name: str
    grades: list[Class]
    weighted_gpa: float | None = None
    unweighted_gpa: float | None = None


def _generate_classes() -> list[Class]:
    return [
        Class(
            title="Math",
            grade=randint(0, 100),
            instructor="Mr. Smith",
            credits=randint(1, 4),
        ),
        Class(
            title="Science",
            grade=randint(0, 100),
            instructor="Mrs. Johnson",
            credits=randint(1, 4),
        ),
        Class(
            title="History",
            grade=randint(0, 100),
            instructor="Mr. Brown",
            credits=randint(1, 4),
        ),
    ]


@mcp.tool(title="Generate Report Card")
def grader_generate_report_card(
    name: str, classes: list[Class] | None = None
) -> ReportCard:
    """
    Genera un reporte completo de calificaciones para un estudiante y una lista de clases.
    Se puede omitir la lista de clases para usar una lista generada aleatoriamente.

    Args:
        name: Nombre del estudiante
        classes: Lista opcional de objetos Class para agregar al reporte
    """
    if not classes:
        classes = _generate_classes()

    weighted_gpa = grader_calculate_gpa(classes)
    unweighted_gpa = grader_calculate_gpa(classes, weighted=False)
    return ReportCard(
        name=name,
        grades=classes,
        weighted_gpa=weighted_gpa,
        unweighted_gpa=unweighted_gpa,
    )


@mcp.tool(
    title="Calculate GPA",
    annotations=ToolAnnotations(readOnlyHint=True),
    structured_output=False,
)
def grader_calculate_gpa(classes: list[Class], weighted: bool = True) -> float:
    """
    Calcula el GPA para una lista de clases. Calcula por defecto el GPA ponderado,
    pero opcionalmente puede calcular el GPA no ponderado.

    Args:
        classes: Lista de clases
        weighted: Indica si se usa GPA ponderado
    """
    if weighted:
        return sum(_class.grade * _class.credits for _class in classes) / sum(
            _class.credits for _class in classes
        )
    return sum(_class.grade for _class in classes) / len(classes)


if __name__ == "__main__":
    # Inicializar y ejecutar el servidor
    mcp.run()
```

En este ejemplo, se definen dos modelos Pydantic: `Class` y `ReportCard`, que servirán como tipos opcionales de entrada y salida para las herramientas.

Se incluye la función `_generate_classes()` para generar aleatoriamente una lista de clases.

La primera herramienta, `grader_generate_report_card()`, es una herramienta con espacio de nombres y nombre legible para humanos “Generate Report Card”, con una descripción detallada. Si no se proporcionan clases, genera una lista de clases y calcula el GPA ponderado y no ponderado usando la herramienta `grader_calculate_gpa()` como función Python normal, retornando el objeto estructurado `ReportCard`. Esta herramienta realiza la acción completa, evitando múltiples llamadas separadas desde la aplicación.

La segunda herramienta, `grader_calculate_gpa()`, es más simple, usa un título legible, una anotación de solo lectura y fuerza que la salida estructurada sea `False`. Toma una lista de objetos `Class` y un booleano para determinar si calcula GPA ponderado o no, realiza el cálculo y retorna un resultado float simple.

El siguiente tipo de primitiva a cubrir serán los **prompts**, que el servidor puede servir a la aplicación host para mejorar la performance en la selección de herramientas.


**Servir Prompts**
Si has trabajado con LLMs por algún tiempo, al menos estarás algo familiarizado con los prompts. Los prompts son las instrucciones que el LLM utiliza para generar su respuesta y típicamente toman la forma de cadenas de texto enviadas al LLM para guiar su respuesta. Para la mayoría de los modelos populares, existe un *system prompt* y un *user prompt*.

El *system prompt* es persistente y guía las respuestas del LLM durante toda la conversación. Normalmente puedes añadir al *system prompt* predeterminado o reemplazarlo completamente por uno personalizado. Si deseas cambiar la personalidad del modelo, indicarle cómo usar herramientas, darle reglas a seguir o proporcionar contexto adicional, puedes hacerlo agregando o reemplazando el *system prompt*.

El *user prompt* generalmente es generado por el usuario y es una consulta directa al modelo, como una pregunta simple o una solicitud de realizar un conjunto específico de acciones. No es persistente y solo se envía al modelo cuando el usuario envía su consulta.

**TIP**
Los *system prompts* son persistentes, lo que significa que se envían al modelo con cada *user prompt* y generan un costo de tokens de entrada. Al diseñar *system prompts*, al igual que con los *user prompts*, debes ser consciente de la eficiencia de tokens y su costo.

En servidores Python MCP, los prompts se representan mediante funciones que retornan ya sea un string, un `UserMessage`, un `AssistantMessage` o una lista de uno o ambos objetos de mensaje representando una conversación parcial, y se decoran con el decorador `@mcp.prompt()`. Desde la perspectiva del servidor, los prompts son esencialmente herramientas con un conjunto restringido de tipos de objetos que pueden retornar y cuyos resultados son manejados de manera especial por la aplicación.

La estructura de las funciones de prompt normalmente usa sus parámetros para inyectar datos de los usuarios de la aplicación en el prompt, aunque también pueden usarse para cualquier valor que tenga sentido incluir, como números que deben ser calculados juntos para inyectar el valor correcto en el prompt. Con MCP, los usuarios de la aplicación o las aplicaciones mismas que usan tu servidor pueden decidir cómo usar los prompts que proporcionas. Si no usas las clases `Message` provistas por el SDK MCP, tus prompts podrían ser usados de formas inesperadas.

---

### Diseñando Prompts

¿Recuerdas la ingeniería de prompts (*prompt engineering*)? Aunque se dijo que sería una disciplina y profesión completamente nueva, la ingeniería de prompts sigue siendo una habilidad valiosa al trabajar con LLMs.

El libro *Prompt Engineering for Generative AI* (O’Reilly, 2024) define los **Cinco Principios de los Prompts**:

1. **Dar dirección**
2. **Especificar formato**
3. **Proporcionar ejemplos**
4. **Evaluar calidad**
5. **Dividir trabajo**

**Dar dirección** es el principio más básico y fundamental. Tus prompts deben dar instrucciones claras al LLM, tanto para guiar la tarea (qué debe hacer) como para guiar el formato (cómo debe responder). Esto se logra proporcionando instrucciones claras, simples y concisas, y listando reglas específicas que el LLM debe seguir. También implica incluir un personaje o ejemplos que mejoren tanto el contenido como el tono de la respuesta del LLM.

**Especificar formato** se refiere típicamente a un formato de salida estructurado, como YAML o JSON, aunque también puede ser un diseño general de la respuesta. Dependiendo del objetivo del prompt, esto puede o no ser necesario. Por ejemplo, si tus prompts solo sirven para llamar con precisión las herramientas del servidor, el formato de salida no importa tanto. Pero si deseas que el prompt genere un documento YAML correctamente formateado, puedes especificar directamente el formato deseado en el prompt, proporcionar ejemplos parciales o completos, o crear una herramienta de conversión YAML en MCP.

**Proporcionar ejemplos** es el tercer principio. Al igual que las personas, los LLMs trabajan mejor cuando tienen algunos ejemplos para aprender. Esto se llama *few-shot learning* y mejora significativamente la precisión y desempeño del modelo. Sin embargo, hay un tradeoff: cuantos más ejemplos proporciones, más limitada y menos creativa será la respuesta del LLM. Debes evaluar lo que realmente deseas de tu prompt y probar su rendimiento con diferentes conjuntos de ejemplos.

**Evaluar calidad** es el cuarto principio. Siempre debes evaluar la calidad de tu prompt ejecutándolo en diversos modelos, especialmente si lo distribuirás a usuarios que podrían usar cualquier modelo gracias a MCP. Puedes usar herramientas como PromptBench o PromptFoo, o realizar pruebas A/B simples con la API de un modelo.

**Dividir trabajo** es el quinto principio. Si notas que tu prompt crece demasiado y trata de hacer más cosas, puede ser momento de dividirlo en prompts separados. Con MCP, también puedes usar técnicas avanzadas como *sampling* para solicitar respuestas al LLM vía el cliente en la aplicación host, y luego proporcionar un prompt final con contexto al usuario, o incluso ejecutar todos los prompts a través del LLM, evitando el modelo típico de interacción del usuario con los prompts MCP.

**Reglas generales adicionales para diseñar prompts:**

* Evita decir lo que no se debe hacer (puede confundir al modelo).
* Usa descripciones precisas (“bastante largo” vs. “2 párrafos de 3-5 oraciones”).
* Utiliza separadores para distinguir entre el contexto y la instrucción dentro del prompt.


**TIP**
La ingeniería de prompts es un tema complejo y matizado, digno de sus propios libros, cursos y otros materiales educativos. Mientras que esta sección cubre lo básico, también deberías revisar *Prompt Engineering for Generative AI* por James Phoenix y Mike Taylor (O’Reilly, 2024) y *Prompt Engineering for LLMs* por John Berryman y Albert Ziegler (O’Reilly, 2024) para convertirte en un experto en ingeniería de prompts.

Como se mencionó al inicio de esta sección, los prompts típicamente se dividen en **system prompts** y **user prompts**. En los modelos principales (como los de OpenAI y Anthropic), estos se representan como mensajes con roles, **system** y **user**, respectivamente. Los mensajes del sistema o **system prompts** especifican la manera en que el modelo debe responder a los mensajes o prompts del usuario, y pueden involucrar cosas como asumir una persona (“you are a helpful assistant”) o proveer directrices o reglas para que el modelo siga (“always respond in a friendly and helpful manner”). Los mensajes o prompts del usuario representan entradas o solicitudes de un usuario (¡y no necesariamente un usuario humano!) al modelo, y son a menudo preguntas directas o solicitudes, como “What are the 50 most popular restaurants in New York City?” Además, estos modelos también tienen un tercer rol, **assistant**, que se usa para representar la respuesta del modelo. Al crear prompts para servidores MCP, el SDK provee las clases `UserMessage` y `AssistantMessage` para representar estos mensajes, facilitando asegurar que tu prompt se use correctamente y permitiéndote crear prompts de múltiples turnos. Una omisión notable en el SDK es la falta de una clase `SystemMessage`; si quieres proveer un **system prompt**, necesitarás devolver un string plano y documentar que el prompt está destinado a ser usado como un **system prompt**.

También existen algunos trucos que puedes usar para mejorar tus prompts que son específicos de modelo, pero que no necesariamente reducen el rendimiento cuando se usan en otros modelos. Dado que los servidores MCP son agnósticos al modelo, es importante no depender demasiado de estos trucos, pero cualquier preocupación sobre rendimiento puede ser aliviada con pruebas exhaustivas. Para los modelos de OpenAI, una de sus principales técnicas de ingeniería de prompts (como se detalla en la documentación de OpenAI) es separar el contexto de las instrucciones del prompt, usando `###` o `"""`. Por ejemplo, el siguiente prompt instruye al modelo a generar una lista de 3 ideas principales a partir del input del usuario colocado debajo. El input del usuario está separado de la instrucción con `###`:

```
Create a list of 3 main ideas from the following text:

###
{user_input}
###
```

En este ejemplo, el bloque de instrucción es “Create a list of 3 main ideas from the following text:”, y el input del usuario se marca como contexto con las vallas `###`. En los modelos de Anthropic, como la familia Claude, se prefieren las etiquetas XML para marcar porciones específicas del prompt. Usar etiquetas XML permite más flexibilidad, ya que puedes marcar cualquier cosa que desees, darle una etiqueta descriptiva, y luego referirte a esa etiqueta más tarde. Como ejemplo simple, aquí está el mismo prompt usando etiquetas XML:

```
<instruction>
Create a list of 3 main ideas from the following text:
</instruction>

<text>
{user_input}
</text>
```

En este ejemplo, en lugar de usar vallas con un único significado, usamos etiquetas XML más expresivas para marcar las porciones de instrucción y texto del prompt. Aunque este ejemplo es simple, puedes imaginar usar múltiples etiquetas XML jerárquicas para contextualizar mejor secciones del prompt, como un recurso de documento con título, preámbulo y cuerpo. Incluso puedes encadenar prompts, solicitando que la respuesta incluya etiquetas, y luego tu siguiente prompt puede referirse a esas etiquetas para el siguiente prompt en la cadena.

Dado que empezamos a mirar prompts reales, es hora de pensar cómo usarlos en tu servidor MCP. Al inicio de esta sección aprendiste que los prompts en MCP son proveídos por funciones, de manera similar a las herramientas. Sin embargo, las funciones de prompt deben devolver un string, un diccionario con las llaves `role` y `message`, un objeto `Message` (o uno de sus hijos), o una secuencia (como una lista) de cualquiera de los tipos listados. Los prompts se registran decorando la función de prompt con `@mcp.prompt()`. Este decorador permite opcionalmente especificar el nombre, título y descripción del prompt, igual que con `@mcp.tool`.

En el siguiente ejemplo, verás 3 funciones de prompt: una que devuelve un string estático, otra que devuelve un f-string con input del usuario, y una que devuelve un objeto `UserMessage` con etiquetas XML.

**1.-Tres funciones de prompt simples que muestran las diferentes maneras de devolver un prompt.**


Ejemplo 5-10.

```python
from mcp.server.fastmcp import FastMCP
from mcp.server.fastmcp.prompts.base import UserMessage

# Inicializar servidor FastMCP
mcp = FastMCP("simple-prompt-server")

@mcp.prompt()
async def simple_string_prompt() -> str:
    """Un prompt simple y estático que saluda al usuario."""
    return "Say hello to the user."

@mcp.prompt()
async def simple_prompt_input(username: str) -> str:
    """Un prompt simple que saluda al usuario por su nombre."""
    return f"Say hello to the user using their name: {username}"

@mcp.prompt()
async def simple_example_prompt(user_text: str) -> UserMessage:
    """Un prompt simple que resume el texto de entrada, usando etiquetas XML."""
    return UserMessage(
        content=f"""
<instruction>
Create a list of 3 main ideas from the following text:
</instruction>

<text>
{user_text}
</text>
    """
    )

if __name__ == "__main__":
    mcp.run()
```

La primera función de prompt, `simple_string_prompt`, no es muy interesante. Simplemente devuelve un string estático que instruye al modelo a saludar al usuario diciendo “Hello”. La segunda es un poco más compleja; toma un argumento `username` y devuelve un string que instruye al modelo a saludar al usuario por su nombre, según el input del usuario. La tercera muestra otra manera de devolver un prompt: creando un objeto `UserMessage` y retornándolo. El string aún tiene interpolación de la entrada del usuario, pero ahora usa etiquetas XML para ayudar al modelo a entender el prompt y lo encierra en un objeto `UserMessage`.

¿Qué pasa si `simple_example_prompt()` devolviera una lista de `UserMessage`s? Esto representaría un *prompt de múltiples turnos*, un prompt que incluye múltiples objetos `Message`, representando una conversación entre el modelo y el usuario. Cada `Message` representa un “turno” en la conversación tomado por uno de los participantes. Cada objeto `Message` tiene un rol, `user` o `assistant`, que se usa para mostrar quién está hablando en ese turno. Esto es poderoso porque permite “precargar” una conversación, guiando aún más fuertemente la respuesta del modelo. El poder real reside en dejar la respuesta del asistente parcialmente completada (una técnica llamada *prefilling*), lo que permite ejercer más control sobre la respuesta del modelo. Por sí solo, el prefilling puede usarse para proporcionar un formato de salida estructurado o para que el modelo mantenga un rol, dependiendo del modelo de interacción con el usuario.

En el siguiente ejemplo, la función de prompt devuelve un prompt de múltiples turnos que prellena la respuesta del asistente con un formato de lista numerada.

1.-Un prompt de múltiples turnos que prellena la respuesta del asistente con un formato de lista numerada.

Ejemplo 5-11.

```python
from mcp.server.fastmcp import FastMCP
from mcp.server.fastmcp.prompts.base import AssistantMessage, UserMessage

# Inicializar servidor FastMCP
mcp = FastMCP("multiturn-prompt-server")

@mcp.prompt()
async def multiturn_prompt(
    main_idea_count: int, user_text: str
) -> list[UserMessage | AssistantMessage]:
    user_input = UserMessage(
        content=f"""
<instruction>
Create a list of {main_idea_count} main ideas from the following text:
</instruction>

<text>
{user_text}
</text>
    """
    )

    assistant_prefill_test = f"""
Here are {main_idea_count} main ideas from the text:
1.
"""
    assistant_prefill = AssistantMessage(content=assistant_prefill_test)
    return [user_input, assistant_prefill]

if __name__ == "__main__":
    mcp.run()
```

En este ejemplo, tenemos la función de prompt `multiturn_prompt()`, que usa el mismo `UserMessage` que `simple_example_prompt()`, pero devuelve una lista de objetos `UserMessage` y `AssistantMessage` para representar una conversación de múltiples turnos. El `UserMessage` se modifica ligeramente para tomar un argumento `main_idea_count`, y el `AssistantMessage` se prellena con una respuesta introductoria y el inicio de una lista numerada. Esto alentará al modelo a continuar la lista, completando la información solicitada.

También puedes usar prompts para controlar el comportamiento del modelo de otras formas. Por ejemplo, es posible incentivar al modelo a priorizar una herramienta al responder a la solicitud de un usuario. Para esto, puedes simplemente tomar la entrada del usuario y añadir una instrucción para verificar la aplicabilidad de cierta herramienta antes que otras. También puedes ser más directo y decirle al modelo que use la herramienta sin importar la solicitud del usuario.

En el siguiente ejemplo, la función de prompt `request_tool_use` toma la solicitud del usuario y le añade una instrucción adicional, indicándole al modelo que use una herramienta específica y agregue el resultado de esa llamada al final de su respuesta. La función `force_tool_use()` fuerza el uso de la herramienta llamándola directamente y prellenando la respuesta del asistente con el resultado.

1.-Un prompt que solicita el uso de una herramienta añadiendo una instrucción adicional a la solicitud del usuario.


In [ ]:
Ejemplo 5-12.

```python
import random

from mcp.server.fastmcp import FastMCP
from mcp.server.fastmcp.prompts.base import AssistantMessage, UserMessage

# Inicializar servidor FastMCP
mcp = FastMCP("tool-use-prompt-server")

@mcp.tool()
async def analyze_sentiment() -> str:
    """Una herramienta que dice la verdad."""
    return random.choice(["positive", "negative", "neutral"])

@mcp.prompt()
async def request_tool_use(user_request: str) -> UserMessage:
    """Un prompt que fuerza al modelo a usar una herramienta."""
    return UserMessage(
        content=f"""
<user_request>
{user_request}
</user_request>
<tool_instruction>
Use the analyze_sentiment tool if available to you to get the sentiment of the
user's request. Respond in such a way to move the user's sentiment to neutral.
</tool_instruction>
    """
    )

@mcp.prompt()
async def force_tool_use(
    user_request: str,
) -> list[UserMessage | AssistantMessage]:
    """Llama directamente a la herramienta y agrega el resultado a la respuesta."""
    user_request_message = UserMessage(content=user_request)
    tool_result = await analyze_sentiment()
    assistant_prefill = AssistantMessage(
        content=f"Your request was {tool_result}, let's "
    )
    return [user_request_message, assistant_prefill]

if __name__ == "__main__":
    mcp.run()
```

En este ejemplo, tenemos una herramienta llamada `analyze_sentiment()`, que devuelve una valoración de sentimiento aleatoria al usuario. La función de prompt `request_tool_use()` toma la solicitud del usuario y añade una instrucción adicional para el modelo, indicándole que use la función y que utilice esos resultados para guiar su respuesta al usuario. Por otro lado, `force_tool_use()` llama directamente a la función de la herramienta y prellena la respuesta del asistente con el resultado y el inicio de la respuesta al usuario. Este código es simple, pero empieza a mostrar cómo se puede influir en el comportamiento de una aplicación combinando prompts con herramientas.

Los recursos, que se cubrirán en la siguiente sección, también pueden usarse dentro de los prompts de MCP. Los recursos son fuentes de datos proporcionadas por tu servidor MCP que tienen un URI especificado por el desarrollador del servidor (tú). Puedes usar este URI en tus prompts para asegurar que el modelo utilice el recurso de alguna manera, por ejemplo, como base para análisis o hechos para una respuesta.

1.-Un ejemplo de función de prompt que usa un URI de recurso para referirse a un recurso en el prompt.

Ejemplo 5-13.

```python
from pathlib import Path

from mcp.server.fastmcp import FastMCP
from mcp.server.fastmcp.prompts.base import UserMessage
from mcp.types import ResourceLink

# Inicializar servidor FastMCP
mcp = FastMCP("basic-resource-server")

@mcp.resource("file://knowledge.txt")
async def knowledge_base() -> str:
    """Un recurso que carga una base de conocimiento basada en texto."""

    # Obtener la ruta absoluta de knowledge.txt relativa a este script
    knowledge_path = Path(__file__).parent / "knowledge.txt"

    with open(knowledge_path, "r") as f:
        return f.read()

@mcp.prompt()
async def knowledge_base_prompt(user_request: str) -> list[UserMessage]:
    """Un prompt que usa el recurso de la base de conocimiento."""
    user_request_message = UserMessage(content=user_request)
    instruction_message = UserMessage(
        content="""
This prompt includes knowledge base from the resource URI: file://knowledge.txt,
please use this resource to answer the user's request. The resource follows
this message:
"""
    )
    resource_message = UserMessage(
        content=ResourceLink(
            name="knowledge_base", uri="file://knowledge.txt", type="resource_link"
        )
    )
    return [user_request_message, instruction_message, resource_message]

if __name__ == "__main__":
    mcp.run()
```

En este servidor, la función `knowledge_base()` está designada como un recurso con el decorador `@mcp.resource()`, y el argumento establece su URI a `file://knowledge.txt`. La función abre el archivo y devuelve su contenido como un string. La función `knowledge_base_prompt()` es la función de prompt, marcada con `@mcp.prompt()`. Toma un argumento `user_request` y construye un prompt de múltiples turnos que incluye la solicitud original del usuario, una instrucción para usar el recurso con su URI, y un `UserMessage` de tipo `ResourceLink` que apunta al URI del recurso. Esta función de prompt agrega el recurso de texto a la solicitud del usuario e instruye al modelo a usar el recurso para ayudar a responder.




# Cómo se usan los prompts

Estos ejemplos muestran cómo servir prompts a host applications, y la magia de MCP significa que como desarrollador de servidor, no necesariamente tienes que preocuparte por cómo los usuarios (aplicaciones host) usan tus prompts. Pero al igual que con herramientas y como verás con recursos, sigue siendo importante entender el **intended user interaction model** para prompts, cómo **las aplicaciones típicamente los usarán**, y cómo el protocolo implementado transfiere los prompts entre el servidor y la aplicación host. La especificación MCP no necesariamente fuerza una manera particular para que los usuarios o aplicaciones interactúen con prompts, pero los prompts fueron diseñados en el protocolo para ser **user-controlled**. Esto significa que el usuario debería poder seleccionar **qué prompts usar y cuándo**, lo que típicamente se implementa como un menú de selección o como **slash commands** en aplicaciones de chat. Entender cómo es probable que tus prompts sean usados te ayudará a diseñarlos de manera que ofrezcan al usuario una **experiencia fluida**.

---

Comprender formas impredecibles en que las aplicaciones pueden usar tus prompts también puede ayudarte a proporcionar una experiencia más flexible para tus usuarios. Por ejemplo:

* Si sabes que las aplicaciones técnicamente pueden usar tus prompts como **mensajes de usuario**, **mensajes de asistente** o incluso **prompts de sistema**, entonces podrías ser empujado a retornar una **lista de objetos `UserMessage`** en lugar de un simple string para darle a la aplicación más información sobre cómo tu prompt fue diseñado para ser usado.
* Tu servidor puede ser usado con aplicaciones que no permiten al usuario ver o seleccionar prompts, como en el Capítulo 3 donde prompts fueron elegidos dinámicamente por el modelo en lugar de por el usuario.
* Tu servidor también puede ser usado por aplicaciones que ponen prompts en el **system prompt**, en lugar de presentarlos como **user** o **assistant messages**. Esto es más complejo de implementar, pero existe, así que vale la pena documentar si tus prompts están pensados para ser usados de esta manera o no, ya sea mediante un **README**, docstrings, o devolviendo explícitamente objetos `UserMessage` y/o `AssistantMessage`.

---

El flujo de mensajes entre el servidor y cliente sigue un patrón general similar al de **tools** y **resources**:

1. **Fase de descubrimiento**:
   Ocurre automáticamente cuando el cliente se conecta al servidor, con el cliente enviando una solicitud `prompts/list` al servidor y el servidor respondiendo con la lista de prompts. Esto normalmente sucede después de que cliente y servidor anuncian que soportan prompts y otros primitives.

2. **Fase de uso**:
   Típicamente se dispara cuando el usuario selecciona un prompt de una lista dentro de la aplicación host. Esto debería causar que el cliente envíe una solicitud `prompts/get` al servidor, y el servidor responderá con el prompt en sí.

3. **Notificaciones durante la conexión**:
   Durante la vida útil de la conexión cliente-servidor, el servidor puede enviar una notificación `prompts/list_changed` al cliente, informándole que la lista de prompts disponibles ha cambiado. El cliente debería entonces responder con una solicitud `prompts/list` al servidor, y el servidor responderá con la lista actualizada de prompts.

> La siguiente figura ilustra cómo estos mensajes de solicitud y respuesta fluyen de ida y vuelta entre cliente y servidor:

---

El siguiente y último **MCP primitive** es el **resource**. Esta sección te dio un adelanto de ellos, y en la siguiente sección entrarás a profundidad en **resources**, cómo funcionan, cómo construirlos y cómo verás a los usuarios utilizándolos en acción.

---

# Servir Recursos

Los **resources** son el último de los tres MCP primitives. Proporcionan **read-only access** a fuentes de datos que pueden actuar como **contexto adicional** para el modelo de lenguaje de la aplicación host, y estas fuentes de datos pueden ser prácticamente cualquier cosa:

* archivos de log
* esquemas de bases de datos
* imágenes
* PDFs
* configuraciones estructuradas (JSON, YAML, etc.)

No estás limitado a estos tipos de fuentes de datos; puedes usar resources para servir cualquier tipo de dato útil para el **host application’s language model**.

---

Los resources se definen con un decorador `mcp.resource()`, que permite especificar:

* el **URI** (requerido, como identificador único o clave para acceder al resource)
* nombre del recurso
* y más

Se recomienda fuertemente incluir **descriptions**, porque pueden ser pasadas al **host application’s language model** por el cliente para ayudarlo a decidir qué recursos, si los hay, usar, ya sea:

* mediante el método mostrado en el Capítulo 3, donde el modelo siempre es solicitado para elegir un recurso, o
* incluyendo **resource URIs** y **descriptions** en un prompt proporcionado por tu servidor

Las **resource annotations** son opcionales y pueden ser proporcionadas a cualquier cliente que use tu servidor para ayudarle a entender cómo usar el resource. Las **resource templates** permiten parametrizar los resources usando **URI templates**, lo cual es crucial para proporcionar fácilmente una amplia gama de diferentes pero relacionados resources. Se profundizará en cada uno de estos temas en “Exposing Resources”.

---

Los resources pueden ser usados de varias maneras, con más usos aún por descubrir por la comunidad, pero todos sirven al mismo objetivo: **proporcionar contexto adicional a un language model para ayudarlo a tomar mejores decisiones y enriquecer sus respuestas**.

En **How Resources Are Used** aprenderás algunas de las maneras en que servidores MCP en producción están utilizando resources:

* análisis de logs
* proporcionando documentación para enriquecer respuestas a coding queries
* cacheando resources en prompt chains
* descubriendo qué sub-resources están disponibles

Verás cómo **sugerir resources para ser usados con una tool**, al igual que el ejemplo ??? en la sección anterior.

---

Primero, examinemos cómo **exponer realmente los resources** a los MCP clients que se conectarán a tu servidor.



# Exponiendo Recursos

Entonces, ahora sabes que un recurso es una fuente de datos que proporciona contexto a un modelo de lenguaje. Pero, ¿cómo se ve eso en la práctica? En el SDK de Python de MCP, los recursos se exponen con funciones de Python decoradas con `@mcp.resource()`. En este decorador, puedes especificar las siguientes propiedades:

### uri

Obligatorio. Un identificador único para el recurso que cumpla con RFC 3986.

### name

Opcional. Un nombre para el recurso, por defecto el nombre de la función.

### title

Opcional. Un nombre legible por humanos para el recurso.

### description

Opcional. Una descripción del recurso, por defecto el docstring de la función.

### mime_type

Opcional. El tipo MIME del recurso a retornar, por defecto `text/plain`.

### icons

Opcional. Una lista de objetos `Icon` que pueden ser usados por el cliente para mostrar el recurso.

**NOTA**
El parámetro `icons` no está documentado en el SDK de Python, pero está disponible para usar con recursos, pero no con plantillas de recursos. Un objeto `Icon` tiene un URI o URL en `src`, un `mimeType` opcional y un string `sizes` opcional.

Aunque los URIs solo deben cumplir con RFC 3986, hay algunos esquemas de URI estándar que la especificación MCP define: `https://`, `file://` y `resource://`. `https://` se debe usar para recursos basados en la web a los que el cliente puede acceder por sí mismo sin asistencia del servidor. `file://` se debe usar para archivos locales y cualquier otra cosa similar a un archivo, incluyendo sistemas de archivos. `git://` también está definido en el protocolo, y se debe usar para denotar recursos git, como un repositorio o commit. MCP también permite crear tus propios esquemas de URI personalizados, siempre que cumplan con RFC 3986.

El cuerpo de la función de prompt debe acceder, leer y devolver el contenido del recurso. Se llama cada vez que el cliente solicita el recurso, por lo que debe mantenerse ligero y eficiente. Debe devolver una cadena para contenido de texto o bytes para blobs binarios, y cualquier otro tipo (como un diccionario) será convertido por el SDK a JSON. En el siguiente ejemplo, verás una función de recurso simple que lee un archivo y devuelve su contenido como una cadena.

**Una función de recurso simple que lee un archivo y devuelve su contenido como una cadena.**

### Example 5-14

```python
from pathlib import Path

from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("basic-resource-server")

@mcp.resource("file://knowledge.txt")
async def knowledge_base() -> Resource:
    """A resource that loads a test-based knowledge base."""

    # Get the absolute path to knowledge.txt relative to this script
    knowledge_path = Path(__file__).parent / "knowledge.txt"

    with open(knowledge_path, "r") as f:
        return f.read()

if __name__ == "__main__":
    mcp.run()
```

Este ejemplo es la misma función de recurso que encontraste en ???, el URI se establece en `file://knowledge.txt`, y la función devuelve la ruta completa al archivo como una cadena. El código que construye la respuesta MCP llamada por el decorador `@mcp.resource()` manejará tu ruta abriendo y leyendo el archivo, devolviendo el contenido como cadena. Para los recursos más simples, esto puede ser todo lo que necesitas. La Figura 5-4 muestra el recurso cargado en MCP Inspector, del cual aprenderás más en [Link to Come]. En esta figura, puedes ver la lista de recursos que el servidor provee en el panel izquierdo, que es solo el recurso `knowledge_base`, y en el panel derecho está el contenido cargado del recurso. Observa cómo se muestran el `uri` y `mimeType`, y debido a que esta función de recurso devuelve una cadena, los contenidos se muestran bajo la clave `text`.

**Figura 5-1.** MCP Inspector con el recurso `knowledge_base` listado en el panel izquierdo y cargado en el derecho.

No estás limitado a URIs codificados directamente para los recursos. Puedes crear **plantillas de recursos**, que son URIs parametrizados que permiten crear un URI de recurso dinámico basado en otra información que puedas tener. Esto es muy similar a un endpoint GET parametrizado en el mundo de la API REST. Puedes crear una plantilla de recurso de la misma manera que creas una función de recurso, pero el URI debe tener variables encerradas entre llaves `{}`. En el siguiente ejemplo, tenemos un servidor que proporciona una plantilla de recurso, dando al cliente el poder de elegir si quiere un archivo u otro.

**1.-Una plantilla de recurso que permite al cliente elegir entre dos archivos.**


### Example 5-15

```python
from pathlib import Path
from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("resource-template-server")

@mcp.resource("file:///{filename}")
async def resource_template(filename: str) -> str | bytes:
    """A resource that loads one of two files based on the filename parameter."""
    # Get the absolute path to the file relative to this script
    file_to_load = Path(__file__).parent / filename

    # Determine if file is binary based on extension
    if file_to_load.suffix.lower() == ".txt":
        with open(file_to_load, "r") as f:
            return f.read()
    else:
        with open(file_to_load, "rb") as f:
            return f.read()

if __name__ == "__main__":
    mcp.run()
```

En este ejemplo, se crea una **plantilla de recurso** con `@mcp.resource()` incluyendo la parte variable del URI del recurso entre llaves `{}`. Observa que hay una `/` extra después del esquema `file://`. Esto se necesita cuando la parte variable del URI está al inicio del URI; de lo contrario, puedes usar `file://` solo. Las partes variables del URI también se convierten en parámetros de la función de recurso, que luego se pueden usar dentro de la función. Aquí se obtiene el objeto `Path` de la ruta completa al archivo, y luego se revisa la extensión del archivo: si es `.txt`, se lee como cadena; de lo contrario, se lee como blob binario. El resultado se devuelve al cliente. Aunque no es necesario para un ejemplo simple, en código listo para producción, la verificación de tipos de archivo debería ser más amplia y robusta. La Figura 5-6 muestra la plantilla de recurso cargada en MCP Inspector con un parámetro de nombre de archivo válido.

---
mcp_inspector_resource_template.png
**Figura 5-2.** MCP Inspector con la plantilla de recurso `resource_template` en el panel central y cargada con un valor de parámetro `2.png` en el panel derecho.

El decorador funciona haciendo que el servidor transforme tu función de recurso en un objeto `FunctionResource`, que luego se agrega a la caché interna del servidor, el **resource manager**. Crear el objeto `FunctionResource` permite almacenar la descripción del recurso y la función en un objeto que se ve como un recurso, de modo que cuando el servidor recibe una solicitud `list resources`, no tiene que cargar los datos de cada recurso. En cambio, la carga del recurso mediante la función se difiere hasta que el cliente lo solicita, mejorando el rendimiento del servidor, especialmente si se sirven recursos grandes.

También puedes exponer recursos directamente con la API de FastMCP, omitiendo el decorador `mcp.resource()` y creando un objeto `Resource` con las propiedades deseadas, y luego llamando a `add_resource()` en la instancia del servidor para añadirlo al resource manager. FastMCP proporciona varias clases derivadas de `Resource` que permiten crear recursos directamente sin usar una función decorada, lo que da más flexibilidad para enriquecer los recursos expuestos al cliente.

---

### Example 5-16

```python
from pathlib import Path
from mcp.server.fastmcp import FastMCP
from mcp.server.fastmcp.resources import FileResource

# Initialize FastMCP server
mcp = FastMCP("resource-object-server")

@mcp.resource("file:///{filename}")
async def resource_template(filename: str) -> FileResource:
    """A resource that loads one of two files based on the filename parameter."""
    # Get the absolute path to the file relative to this script
    file_to_load = Path(__file__).parent / filename
    if file_to_load.suffix.lower() == ".txt":
        binary_flag = False
    else:
        binary_flag = True
    return FileResource(
        uri=f"file:///{filename}", path=file_to_load, is_binary=binary_flag
    )

filename = "1.txt"
file_resource = FileResource(
    uri=f"file:///{filename}", path=Path(__file__).parent / filename
)
mcp.add_resource(file_resource)

if __name__ == "__main__":
    mcp.run()
```

En este ejemplo, se modifica la función `resource_template()` del ejemplo anterior para devolver un objeto `FileResource` en lugar de una cadena o bytes. Se mantiene la verificación simple de la extensión del archivo, solo para establecer un booleano que se usa en el parámetro `is_binary` del constructor `FileResource`. También se muestra cómo crear un objeto `FileResource` directamente en el cuerpo principal del código del servidor y añadirlo manualmente al cache del resource manager con `mcp.add_resource()`. Generalmente, solo deberías usar el decorador `@mcp.resource()` para crear recursos, pero si necesitas algo más especializado, una clase derivada de `Resource` podría ser adecuada.

**TIP:** FastMCP proporciona varias clases de conveniencia para recursos, incluyendo `FileResource`, `HttpResource`, `FunctionResource` y más. Estas clases se encuentran en el módulo `mcp.server.fastmcp.resources` o en el repositorio de GitHub del SDK de Python.

También puedes **anotar tus recursos y plantillas de recursos**, proporcionando información adicional al cliente. El protocolo define las claves de anotación aceptadas como:

* `audience`: un array JSON de strings con valores `user`, `assistant` o `both`.
* `priority`: un rango de 0.0 a 1.0 indicando la importancia del recurso.
* `lastModified`: almacena la marca de tiempo de la última modificación del recurso.

Desafortunadamente, la versión de FastMCP incluida con el SDK de Python de MCP no soporta estas anotaciones. Para usarlas, necesitarás FastMCP 2 (2.11 o posterior) y añadirlas como parámetro al decorador, o usar el servidor de bajo nivel del SDK estándar de Python, donde puedes devolver un objeto `Resource` con el campo `annotations`.

En los dos ejemplos anteriores, viste dos tipos de contenido devueltos: **texto** y **blobs binarios**. El texto se utiliza para cualquier recurso basado en texto, como archivos de configuración, logs o datos JSON. Los blobs binarios se usan típicamente para imágenes, audio, PDFs o cualquier contenido que no sea texto y se devuelven como objetos `bytes`. Todo lo que se devuelva en una función de recurso o plantilla de recurso que no sea cadena o bytes será convertido a JSON y enviado al cliente.

### ADVERTENCIA

Ten cuidado con los **antipatrones** al desarrollar recursos. Los recursos deben ser **ligeros** y evitar cálculos pesados dentro de sus funciones. Del mismo modo, las funciones de recursos no deben tener **efectos secundarios** y no deben utilizarse para disparar otras acciones u operaciones. Todo esto es posible porque el acceso a recursos se mediatiza mediante funciones de Python, por lo que técnicamente se podrían usar para más de lo que fueron diseñadas. Resiste esa tentación y mantén tus funciones de recurso ligeras y enfocadas únicamente en entregar el recurso al cliente.

---

### Bajo el capó

En el servidor de bajo nivel, el protocolo define varias acciones para los recursos:

* **Listar recursos:** se activa con una solicitud `resources/list` al servidor, que responde con una lista de recursos incluyendo URI, nombre, título, descripción y `mimeType` de cada recurso disponible. FastMCP maneja esto automáticamente, similar a las respuestas análogas para herramientas y prompts. Si usas la API de bajo nivel, deberás implementar esto manualmente.
* **Listar plantillas de recursos:** se hace con `resources/templates/list` y funciona igual que `resources/list`.
* **Leer un recurso:** se realiza con `resources/read` incluyendo un URI para que el servidor devuelva el recurso correcto. En el SDK de Python, la función de recurso se llama para recuperar el recurso.
* **Suscribirse a recursos:** los clientes pueden suscribirse a recursos individuales y recibir notificaciones cuando estos cambien.

---

### Cómo se usan los recursos

Los recursos se usan de varias maneras, siempre con el objetivo de **proporcionar contexto adicional al modelo de lenguaje de la aplicación host**, ayudando a tomar mejores decisiones y comprender mejor su entorno. Algunos usos comunes incluyen:

* **Análisis de logs:** usar plantillas de recursos para seleccionar subconjuntos de logs que se entregan a un agente de análisis o de soporte.
* **Documentación de código:** servidores MCP como AWS MCP y Context7 entregan documentación de código actualizada al modelo, mejorando la calidad de las respuestas de asistentes de programación.

Otros usos más avanzados incluyen:

* **Descubrimiento de entidades:** útil si tienes muchos recursos que descubrir o si los recursos disponibles son dinámicos y dependen del usuario o aplicación. En este patrón, un recurso devuelve un diccionario de la entidad que se está construyendo, con su ID interno como clave y un diccionario de propiedades como valor, ayudando al modelo a decidir qué entidad usar. Esta lista puede añadirse al prompt del usuario como `UserMessage` para que el modelo elija la entidad correcta, que luego se pasa como parámetro a una herramienta de consulta.

* **Caché de referencias de recursos:** evita que los usuarios saturen el contexto del prompt con recursos innecesarios. El patrón suele implementarse del lado del cliente. El usuario realiza una operación tipo **RAG** obteniendo recursos del servidor, que devuelve una lista de URIs. El cliente añade estas URIs al prompt con etiquetas XML que incluyen URI y descripción, y para elementos nuevos, su texto completo. El XML comprimido también se pasa al modelo y, si el modelo decide usar el recurso, la aplicación lo obtiene desde la caché. El proyecto *tupac*, vinculado en el Capítulo 3, implementa este patrón para cachear recursos en el contexto del prompt.

---

### Control de los recursos

Los recursos son **controlados por la aplicación**, es decir, depende de la aplicación host decidir cómo integrarlos en el contexto del modelo.

**Flujo de mensajes:**

1. El cliente envía `resources/list` al servidor.
2. El servidor responde con la lista de recursos disponibles.
3. Para usar un recurso, el cliente envía `resources/read` con el URI correspondiente.
4. El servidor devuelve el contenido del recurso.

**Suscripciones:**

1. El cliente envía `requests/subscribe`.
2. El servidor confirma la suscripción.
3. Notificaciones de actualización son iniciadas por el servidor. Cuando un recurso se actualiza, el servidor envía `notifications/resources/updates`, y el cliente responde con `resources/read` para obtener el recurso actualizado.

---

Este capítulo cubrió los usos básicos de los tres **primitivos MCP**: **tools**, **prompts** y **resources**. Aprendiste:

* Para qué se usan los servidores en el ecosistema MCP.
* Cómo conectarlos a clientes de distintas maneras.
* Las dos APIs del SDK de Python: **FastMCP** y la **API de bajo nivel**.
* Cómo diseñar y usar herramientas y prompts, y cómo exponer recursos.
* Los patrones de interacción cliente-servidor para cada primitivo, incluyendo usos comunes y creativos.

Estos conocimientos pueden inspirarte para crear nuevos casos de uso al construir tus propios servidores MCP, o convertirse en herramientas indispensables en tu caja de herramientas para agentes.

---

En el próximo capítulo, se profundizará en **desarrollo de servidores**, incluyendo:

* Utilidades del servidor como **completions**, **logging**, **notifications** y **pagination**.
* Uso de **recursos proporcionados por el cliente**, como **elicitations** para obtener entrada del usuario, **sampling** para consultar el modelo de la aplicación host y **roots** para marcar dónde en el sistema de archivos host el servidor puede acceder.
